In [ ]:
# Sources used:
# https://math.stackexchange.com/questions/945871/derivative-of-softmax-loss-function
# https://medium.com/usf-msds/deep-learning-best-practices-1-weight-initialization-14e5c0295b94

import numpy as np
import matplotlib.pyplot as plt


class NeuralNetwork:
    def __init__(self, layers, relu_alpha=0.01, std=0.1):
        """
        Constructor.

        Args:
            layers: Number of neurons for each layer.
            relu_alpha: Parameter of leaky ReLU (set to 0 for default ReLU behavoiur).

        Returns:
            New NerualNetwork instance.
        """
        self.layers = layers
        self.relu_alpha = relu_alpha
        self.W = [None] * (len(layers) - 1)
        self.dW = [None] * (len(layers) - 1)
        self.b = [None] * (len(layers) - 1)
        self.db = [None] * (len(layers) - 1)
        self.z = [None] * len(layers)
        self.a = [None] * len(layers)
        
        for i in range(1, len(layers)):
            self.W[i - 1] = np.random.randn(layers[i - 1], layers[i]) * std
            self.b[i - 1] = np.random.randn(1, layers[i]) * std

        
    def forward(self, X):
        """
        Runs network.

        Args:
            X: Input with shape==(N, D) (N: number of input rows, D: input dimension).

        Returns:
            Softmax probabilities of classes.
        """
        self.a[0] = X
        
        for i in range(len(self.W)):
            self.z[i + 1] = self.a[i].dot(self.W[i]) + self.b[i]
            
            if i == len(self.W) - 1:
                self.probs = np.e**self.z[i + 1]
                self.probs = self.probs / np.sum(self.probs, axis=1, keepdims=True)
                return self.probs
            
            self.a[i + 1] = self.z[i + 1].copy()
            self.a[i + 1][self.a[i + 1] < 0] *= self.relu_alpha
    
    
    def backward(self, y):
        """
        Computes gradient based on previous forward run.

        Args:
            y: One-hot encoded target output labels.

        Returns:
            None.
        """
        dz = (self.probs - y) / len(y)
        
        for i in reversed(range(len(self.W))):
            self.dW[i] = self.a[i].T.dot(dz)
            self.db[i] = dz.sum(axis=0, keepdims=True)
            
            if i == 0:
                break

            da = dz.dot(self.W[i].T)
            dz = da
            dz[self.z[i] < 0] *= self.relu_alpha
        
        
    def step(self, lr):
        """
        Performs one step of gradient descent.

        Args:
            lr: Learning rate.

        Returns:
            None.
        """
        for i in range(len(self.W)):
            self.W[i] -= self.dW[i] * lr
            self.b[i] -= self.db[i] * lr


    def check_gradient(self, X, y):
        """
        Checks gradient by comparing it to numerical gradient.

        Args:
            X: Input with shape==(N, D) (N: number of input rows, D: input dimension).
            y: One-hot encoded target output labels.

        Returns:
            None.
        """
        h = 10**-6
        threshold = 10**-6
        
        for k in range(len(self.W)):
            for i in range(self.W[k].shape[0]):
                for j in range(self.W[k].shape[1]):
                    old = self.W[k][i, j]
                    self.W[k][i, j] = old - h
                    probs = self.forward(X)
                    loss1 = -np.log((probs * y).sum(axis=1)).mean()
                    
                    self.W[k][i, j] = old + h
                    probs = self.forward(X)
                    loss2 = -np.log((probs * y).sum(axis=1)).mean()
                    
                    grad = (loss2 - loss1) / (2 * h)
                    print(f'dW {grad - self.dW[k][i, j]}')
                    self.W[k][i, j] = old
            
            for j in range(self.b[k].shape[1]):
                old = self.b[k][0, j]
                self.b[k][0, j] = old - h
                probs = self.forward(X)
                loss1 = -np.log((probs * y).sum(axis=1)).mean()

                self.b[k][0, j] = old + h
                probs = self.forward(X)
                loss2 = -np.log((probs * y).sum(axis=1)).mean()

                grad = (loss2 - loss1) / (2 * h)
                print(f'db {grad - self.db[k][0, j]}')
                self.b[k][0, j] = old


D = 2 # Input dimension
K = 3 # Number of classes
KN = 100 # Number of instances per class
N = K * KN # Total number of instances

X = np.zeros((N, D))
y = np.zeros((N, K))

for i in range(K):
    angle = np.random.uniform(0, 2 * np.pi, (KN,))
    noise = np.random.randn(KN) * 0.2
    X[i*KN:(i+1)*KN, 0] = np.cos(angle) * (i + 0.4 + noise)
    X[i*KN:(i+1)*KN, 1] = np.sin(angle) * (i + 0.4 + noise)
    y[i*KN:(i+1)*KN, i] = 1
    
plt.axes().set_aspect('equal', 'datalim')
plt.plot(X[:KN, 0], X[:KN, 1], 'ro')
plt.plot(X[KN:2*KN, 0], X[KN:2*KN, 1], 'go')
plt.plot(X[2*KN:, 0], X[2*KN:, 1], 'bo')

nn = NeuralNetwork([2, 8, 3])

for epoch in range(100):
    probs = nn.forward(X)
    loss = -np.log((probs * y).sum(axis=1)).mean()
    print(f'loss: {loss}')
    nn.backward(y)
#     nn.check_gradient(X, y)
    nn.step(0.5)